## S-Mobile: Predicting Customer Churn
* Team-lead GitLab id: 243
* Group name: Korinna
* Team member names: Siqi Chen, Wen-Hsuan Hung, Xinyu Lou, Yuefeng Mao

## Setup

Please complete this Jupyter notebook by answering the questions in `s-mobile.pdf` on Canvas (week9/). Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML from the Jupyter Notebook file without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the fourth group assignment for MGTA 455 and you will be using git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". git will not decide for you who's change to accept so the team-lead will have to determine which edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and then push them to GitLab. Make "pull first" a habit!

In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import os
import urllib.request
import zipfile
from tempfile import NamedTemporaryFile as tmpfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

import statsmodels.formula.api as smf
from sklearn import preprocessing
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import  OneHotEncoder

from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, make_scorer,mean_squared_error,r2_score
from sklearn.inspection import plot_partial_dependence 

import statsmodels.formula.api as smf 
from statsmodels.genmod.families import Binomial 
from statsmodels.genmod.families.links import logit 

import seaborn as sns

# increase plot resolution
mpl.rcParams["figure.dpi"] = 100

In [2]:
## load the data - this dataset must NOT be changed
s_mobile = pd.read_pickle("data/s_mobile.pkl")
s_mobile["churn_yes"] = rsm.ifelse(s_mobile["churn"] == "yes", 1, 0)

If you want access to the full 1M row dataset, use the code below to download and use the data. Please do **not** include the 1M row dataset in your repo!

The downside to using the dataset with 1M rows is, of course, that estimation time will increase substantially. I do NOT recommend you use this dataset to select your final model or for tuning hyper parameters. You can, however, use this larger dataset to re-estimate your chosen model and generate profit estimates for the representative sample.

In [3]:
## uncomment to run
# url = "https://www.dropbox.com/s/xhiexneeok9gyhs/s_mobile_1M.pkl.zip?dl=1"
# file_path, _ = urllib.request.urlretrieve(url)
# zip_file = zipfile.ZipFile(file_path, "r")
# s_mobile_tmp = zip_file.open(zip_file.namelist()[0])
# os.remove(file_path)
# s_mobile = pd.read_pickle(s_mobile_tmp)

In [4]:
# show dataset description
rsm.describe(s_mobile)

## S-mobile

Dataset used to investigate opportunities to decrease customer churn at S-mobile. The sample consists of three parts:

1. A training sample with 27,300 observations and a 50% churn rate ("training == 1")
2. A test sample with 11,700 observations and a 50% churn rate ("training == 0")
3. A representative sample with 30,000 observations and a churn rate of 2%, i.e., the actual monthly churn rate for S-mobile ("is.na(training)" or "representative == 1")

## Variables

* customer: Customer ID
* churn: Did consumer churn in the last 30 days? (yes or no)
* changer: % change in revenue over the most recent 4 month period
* changem: % change in minutes of use over the most recent 4 month period
* revenue: Mean monthly revenue in SGD
* mou: Mean monthly minutes of use
* overage: Mean monthly overage minutes
* roam: Mean number of roaming calls
* conference: Mean number of conference calls
* months: # of months the customer has had service with S-Mobile
* uniqsubs: Number of individuals listed on the customer account
* custcare: Mean number of calls to customer care 
* retcalls: Number of calls by the customer to the retention team
* dropvce: Mean number of dropped voice calls 
* eqpdays: Number of days customer has owned current handset
* refurb: Handset is refurbished (no or yes)
* smartphone: Handset is a smartphone (no or yes)
* creditr: High credit rating as opposed to medium or low (no or yes)
* mcycle: Subscriber owns a motorcycle (no or yes)
* car: Subscriber owns a car (no or yes)
* travel: Subscriber has traveled internationally (no or yes)
* region: Regions delineated by the 5 Community Development Council Districts (e.g., CS is Central Singapore)
* occupation: Categorical variable with 4 occupation levels (professional, student, retired, or other)
* training: 1 for training sample, 0 for test sample, NA for representative sample
* representative: 1 for representative sample, 0 for training and test sample


Use `smf.glm` with `freq_weights` and `cov_type` like in the below example
    
```python
lr = smf.glm(
    formula="churn_yes ~ changer + changem + ...",
    family=Binomial(link=logit()),
    data=pentathlon_nptb.query("training == 1"),
    freq_weights=s_mobile.loc[mobile.training == 1, "cweight"],
).fit(cov_type="HC1")
```

In [5]:
# run python code from another notebook
%run ./sub-notebooks/model1.ipynb

     index     OR   OR%  2.5%  97.5% p.values    
1  changer  1.001  0.1%   1.0  1.002    0.006  **


In [6]:
# importing functions from a module/package
from utils import functions

functions.example()


You just accessed a function from your first python packages!
Change the code in utils/function.py to whatever you need for this assignment
Use 'from utils import functions' to get access to your code
You can add modules to import from by adding additional .py files to the 'utils' directory
Note: If you make changes to the content of this file you will have to restart the notebook kernel to get the updates



In [7]:
pd.options.mode.chained_assignment = None

## Data Pre-processing

In [8]:
s_mobile["cweight"] = rsm.ifelse(s_mobile.churn == "yes", 2, 98)

In [9]:
to_std = s_mobile.loc[:, "changer":"eqpdays"].columns

# scale numeric variables by (x - mean(x)) / sd(x)
s_mobile_std = s_mobile.copy()
s_mobile_std[to_std] = rsm.scale_df(
   s_mobile[to_std], sf=1
)

In [10]:
s_mobile_train = s_mobile[s_mobile['training'] == 1]
s_mobile_test = s_mobile[s_mobile['training'] == 0]

## Model comparison: Grid search

In [16]:
columns = ['refurb' , 'smartphone' , 'highcreditr' , 'mcycle' , 'car' ,'travel' , 'region' , 'occupation']

X_train_catagroy= pd.get_dummies(s_mobile_std[columns])
X_num = s_mobile_std.loc[:,"changer":"eqpdays"]
X_index = s_mobile_std.loc[:,"training":"representative"]

X = pd.concat([X_train_catagroy,X_num,X_index],axis=1)

X_train = X[X['training']==1]
X_test = X[X['training']==0]
X_repre = X[X['representative']==1]

X_train = X_train.drop(['training','representative'],axis=1)
X_test =  X_test.drop(['training','representative'],axis=1)
X_repre = X_repre.drop(['training','representative'],axis=1)

Y_train = s_mobile_std[s_mobile_std['training']==1]["churn_yes"]
Y_test = s_mobile_std[s_mobile_std['training']==0]["churn_yes"]
Y_repre = s_mobile_std[s_mobile_std['representative']==1]["churn_yes"]

### GradientBoosting

In [20]:
# Grid search
learning_rate = list(np.arange(0.4,0.6,0.05))
n_estimators = range(50,110,10)
max_depth = range(1,5)

def gridsearch_gb(learning_rate,n_estimators,max_depth):
    params_scores = {}
    for j in n_estimators:
        for k in max_depth:
            for g in learning_rate:
                model = GradientBoostingClassifier(n_estimators=j,max_depth=k,learning_rate=g).fit(X_train,Y_train)
                y_pred = model.predict_proba(X_test)[:,1]
                score = roc_auc_score(Y_test, y_pred)
                params_scores[(j,k,g)] = score
                print("n_estimators=",j,"max_depth=",k,"learning_rate=",g," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_gb(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate)

n_estimators= 50 max_depth= 1 learning_rate= 0.4  AUROC on testing set= 0.7238125210022646
n_estimators= 50 max_depth= 1 learning_rate= 0.45  AUROC on testing set= 0.727327430783841
n_estimators= 50 max_depth= 1 learning_rate= 0.5  AUROC on testing set= 0.728751654613193
n_estimators= 50 max_depth= 1 learning_rate= 0.55  AUROC on testing set= 0.7299038351961429
n_estimators= 50 max_depth= 2 learning_rate= 0.4  AUROC on testing set= 0.7486488567462927
n_estimators= 50 max_depth= 2 learning_rate= 0.45  AUROC on testing set= 0.7461704288114546
n_estimators= 50 max_depth= 2 learning_rate= 0.5  AUROC on testing set= 0.7519846007743444
n_estimators= 50 max_depth= 2 learning_rate= 0.55  AUROC on testing set= 0.7522140550807218
n_estimators= 50 max_depth= 3 learning_rate= 0.4  AUROC on testing set= 0.7570113521805829
n_estimators= 50 max_depth= 3 learning_rate= 0.45  AUROC on testing set= 0.7556136021623201
n_estimators= 50 max_depth= 3 learning_rate= 0.5  AUROC on testing set= 0.7553255168383

(100, 3, 0.4)

### Random Forest

In [22]:
n_estimators=range(10,110,10)
max_depth=range(2,30,2)

def gridsearch_rf(n_estimators,max_depth):
    params_scores = {}
    for j in n_estimators:
        for k in max_depth:
            model = RandomForestClassifier(n_estimators=j,max_depth=k).fit(X_train,Y_train)
            y_pred = model.predict_proba(X_test)[:,1]
            score = roc_auc_score(Y_test, y_pred)
            params_scores[(j,k)] = score
            print("n_estimators=",j,"max_depth=",k," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_rf(n_estimators=n_estimators,max_depth=max_depth)

n_estimators= 10 max_depth= 2  AUROC on testing set= 0.6818138943677405
n_estimators= 10 max_depth= 4  AUROC on testing set= 0.6953303820585872
n_estimators= 10 max_depth= 6  AUROC on testing set= 0.7133400394477317
n_estimators= 10 max_depth= 8  AUROC on testing set= 0.7175716414639493
n_estimators= 10 max_depth= 10  AUROC on testing set= 0.7174326247351888
n_estimators= 10 max_depth= 12  AUROC on testing set= 0.7193188983855651
n_estimators= 10 max_depth= 14  AUROC on testing set= 0.7140423990065016
n_estimators= 10 max_depth= 16  AUROC on testing set= 0.7080802688289868
n_estimators= 10 max_depth= 18  AUROC on testing set= 0.7071766820074512
n_estimators= 10 max_depth= 20  AUROC on testing set= 0.692428563079845
n_estimators= 10 max_depth= 22  AUROC on testing set= 0.6983472569216158
n_estimators= 10 max_depth= 24  AUROC on testing set= 0.6950752428957558
n_estimators= 10 max_depth= 26  AUROC on testing set= 0.6900980056980056
n_estimators= 10 max_depth= 28  AUROC on testing set= 0.

(100, 14)